# 인터파크

In [1]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# from urlib는 안하셨네? 

- 순위, 책제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

### 1. 데이터 가져오기 

In [3]:
site = 'http://book.interpark.com'
url = 'http://book.interpark.com/display/collectlist.do?_method=bestsellerHourNew&bookblockname=b_gnb&booklinkname=%BA%A3%BD%BA%C6%AE%C1%B8&bid1=w_bgnb&bid2=LiveRanking&bid3=main&bid4=001'
req = requests.get(url)
html = req.text

In [4]:
soup = BeautifulSoup(html, 'html.parser')

### 2. 찾으려는 데이터의 태그 찾기

In [5]:
lis = soup.select_one('.rankBestContentList ').find_all('li')
len(lis)


67

In [6]:
lis = soup.select('.rankBestContentList > ol > li' )
len(lis)


15

### 3. 하나의 데이터 선택해서 원하는 정보추출

- 순위, 제목, 저자, 출판사, 가격(정수), 발행일, 쪽수(정수)

- 트렌드 코리아 2022

In [7]:
li = lis[0]
href = li.find('a')['href']
href
# href이 뭘 위한 건지 잘 모르겠다. 



'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [8]:
sub_url = site + href 
sub_url

'http://book.interpark.com/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=354069935&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [9]:
sub_req = requests.get(sub_url)
sub_html = sub_req.text
sub_soup = BeautifulSoup(sub_html, 'html.parser')

- rank 구하기

- 1 ~ 9 위

In [14]:
li.select_one('.rankBtn_ctrl')

<span class="rankBtn_ctrl rkNum_B01"></span>

In [19]:
li.select_one('.rankBtn_ctrl')['class']

['rankBtn_ctrl', 'rkNum_B01']

In [20]:
li.select_one('.rankBtn_ctrl')['class'][1][-1]

'1'

- 10 ~ 15 위

In [24]:
li = lis[12]        # rank 구성이 두 개 나오는 걸 확인하는 과정이 있어야 함! 
li.select('.rankBtn_ctrl')      

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M02"></span>]

In [28]:
rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
rank

'1'

In [29]:
rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
rank

'12'

- 모든 경우에 대처

In [30]:
if len(li.select('.rankBtn_ctrl'))== 1:
    rank = li.select_one('.rankBtn_ctrl')['class'][1][-1]
    rank = int(rank)
else:
    rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
    rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
    rank = int(rank)
rank

        

12

In [ ]:
title = li.select_one('.itemName').get_text().strip()
title



'트렌드 코리아 2022'

In [ ]:
author = li.select_one('.author').get_text().strip()
author

'김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영'

In [ ]:
pub = li.select_one('.company').get_text().strip()
pub

'인플루엔셜'

In [82]:
price = li.select_one('.price').find('em').get_text().strip()
price = int(price.replace(',',''))
price

14220

- 출판일

In [33]:
date = sub_soup.select_one('.bInfo_txt').get_text().split('\n')[3]
date

'발행 : 2021년 10월 06일'

In [44]:
sub_lis = sub_soup.select('.bInfo_txt > li')

In [45]:
date = sub_lis[-3].get_text().split(':')[1].strip()
date

'2021년 10월 06일'

In [50]:
import re
re.sub('[^0-9]','', date)       # sub : 대체

'20211006'

- 쪽수

In [51]:
# 출판일자와 같은 과정으로 다시 
page = int(sub_soup.select_one('.bInfo_txt').get_text().split('\n')[4][5:])
page

452

In [70]:
sub_lis = sub_soup.select('.bInfo_txt > li')
page = sub_lis[-2].get_text().split(':')[1].strip()
page


'452'

### 4. 페이지에 있는 전체 데이터 가져오기

- with 제목, 저자, 출판사, 가격 

In [83]:
lines = []
for li in lis:

    if len(li.select('.rankBtn_ctrl'))== 1:
        rank = li.select_one('.rankBtn_ctrl')['class'][1][-1]
        rank = int(rank)
    else:
        rank = li.select('.rankBtn_ctrl')[0]['class'][1][-1]
        rank += li.select('.rankBtn_ctrl')[1]['class'][1][-1]
        rank = int(rank)
    rank

    title = li.select_one('.itemName').get_text().strip()
    author = li.select_one('.author').get_text().strip()
    pub = li.select_one('.company').get_text().strip()
    price = li.select_one('.price').find('em').get_text().strip()
    price = int(price.replace(',',''))
    
    sub_url = site + href 
    sub_req = requests.get(sub_url)
    sub_html = sub_req.text
    sub_soup = BeautifulSoup(sub_html, 'html.parser')
    sub_lis = sub_soup.select('.bInfo_txt > li')
    date = sub_lis[-3].get_text().split(':')[1].strip()
    date = re.sub('[^0-9]','', date)
    page = int(sub_lis[-2].get_text().split(':')[1].strip())
    
    
    lines.append([rank, title, author, pub, price, date, page])

df = pd.DataFrame(lines, columns =['순위', '제목', '저자', '출판사', '가격', '출판일', '쪽수'] )
df


,순위,제목,저자,출판사,가격,출판일,쪽수
0,1,트렌드 코리아 2022,"김난도(金蘭都), 전미영, 최지혜, 이향은, 이준영",미래의창,16200,20211006,452
1,2,2022 현직교사들이 들려주는 면접레시피,"류은진, 양왕경, 이광한, 이지혜, 양왕경, 이광한, 이지혜",미래가치,42300,20211006,452
2,3,아직 오지 않은 날들을 위하여,파스칼 브뤼크네르/이세진 역,인플루엔셜,14400,20211006,452
3,4,거인의 포트폴리오,강환국,페이지2북스,16200,20211006,452
4,5,불편한 편의점(15만부 기념 윈터 에디션),김호연,나무옆의자,12600,20211006,452
5,6,거꾸로 읽는 세계사,유시민(柳時敏),돌베개,15750,20211006,452
6,6,달러구트 꿈 백화점,이미예,팩토리나인,12420,20211006,452
7,8,작은 별이지만 빛나고 있어(10만 부 기념 한정판 에디션),소윤,북로망스,13500,20211006,452
8,9,달러구트 꿈 백화점. 2,이미예,팩토리나인,12420,20211006,452
9,10,오십에 읽는 논어,최종엽,유노북스,14400,20211006,452


In [ ]:
df.to_csv('인터파크_베셀(211207).csv', idex =False) 나중에 저장하기 다 풀고!! 